In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [2]:
# Read the data from CSV files
temp = pd.read_csv('../For_preprocessing.csv')
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768 entries, 0 to 4767
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4763 non-null   object
 1   Sentiment    4766 non-null   object
dtypes: object(2)
memory usage: 74.6+ KB


In [3]:
temp=temp.dropna(subset=['Review_Text'])
temp=temp.dropna(subset=['Sentiment'])
temp = temp.reset_index(drop=True)

temp_val = temp['Sentiment'].tolist()
#len(temp_val)
ind_list = []

for i in range(len(temp['Sentiment'])):
    if temp_val[i] != '1' and temp_val[i] != '2' and temp_val[i] != '3':
        ind_list.append(i)
        
temp.reset_index(drop=True)
temp.drop(temp.index[ind_list], inplace=True)

In [4]:
temp = temp.reset_index(drop=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4699 non-null   object
 1   Sentiment    4699 non-null   object
dtypes: object(2)
memory usage: 73.5+ KB


In [5]:
st = temp['Sentiment'].tolist()
st=set(st)
st

{'1', '2', '3'}

In [ ]:
# # Function to normalise hotel ratings in the range of -1 to 1

# def change_labels(r):
#     if(r=='1'):
#         return '0'
#     if(r=='2'):
#         return '1'
#     if(r=='3'):
#         return '2' 

In [ ]:
# temp['Sentiment'] = temp['Sentiment'].apply(lambda x:change_labels(x))
# temp

In [6]:
temp_val = temp['Sentiment'].tolist()
#len(temp_val)
ind_list = []

for i in range(len(temp['Sentiment'])):
    if temp_val[i] != '1' and temp_val[i] != '2' and temp_val[i] != '3':
        ind_list.append(i)
temp.reset_index(drop=True)
temp.drop(temp.index[ind_list], inplace=True)

In [7]:
scale_mapper3 = {"1":0, "2":1, "3":2}
temp['Sentiment'] = temp['Sentiment'].replace(scale_mapper3)

In [8]:
temp = temp.reset_index(drop=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4699 non-null   object
 1   Sentiment    4699 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.5+ KB


In [9]:
temp

Review_Text  Sentiment
0     Its really nice place to stay especially for b...          2
1     It seems that hotel does not check the basic a...          0
2     Worst hotel I have ever encountered. I will ne...          0
3     Had a good time in this hotel and the staff Ku...          2
4     good hotel and staff Veg food good non veg bre...          2
...                                                 ...        ...
4694  My fifth stay at the hotel for business. Rooms...          2
4695                                          enjoyable          2
4696  Most impressive service by staff in all areas....          2
4697  The linens were smelling bad, and the elevator...          0
4698  Stayed at the Hyatt in Chennai for the first t...          2

[4699 rows x 2 columns]

In [ ]:
def tokenize_reg():
    temp['Review_Text'] = temp['Review_Text'].replace(r'http\S+','',regex=True).replace(r'www\S+','',regex=True).replace(r'\d+','',regex=True)
    tokens = RegexpTokenizer(r'\w+')
    temp['Review_Text']=temp['Review_Text'].apply(lambda x:tokens.tokenize(x.lower()))
    stop_words = set(stopwords.words('english')) 
    stop_words = stop_words.union(",","(",")","[","]","{","}","#","@","!",":",";",".","?")
    temp['Review_Text'] = temp['Review_Text'].apply(lambda x: [item for item in x if item not in stop_words])
tokenize_reg()

In [ ]:
lem = WordNetLemmatizer()
def lemm(text):
    sent=[]
    for word in text:
        sent.append(lem.lemmatize(word))
    return sent

ps=PorterStemmer()

def stemm(text):        
    sent = []
    for word in text:
        sent.append(ps.stem(word))
    return sent

temp['Review_Text'] =  temp.apply(lambda x: stemm(lemm(x['Review_Text'])), axis=1)

In [ ]:
revs=temp['Review_Text'].tolist()
for i in range(len(revs)):
    revs[i]=' '.join(revs[i])
temp['Review_Text'] = revs
temp

In [ ]:
pd.to_numeric(temp['Sentiment'])
st = temp['Sentiment'].tolist()
st=set(st)

In [ ]:
temp.info()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier


text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', XGBClassifier(use_label_encoder=False,
                                  eval_metric='mlogloss'))])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__min_child_weight': [1, 5, 10],
        'clf__gamma': [0.5, 1, 1.5, 2, 5],
        'clf__subsample': [0.2, 0.4, 0.6, 0.8, 1.0],
        'clf__colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1.0],
        'clf__max_depth': [3, 4, 5,6,8,10,12, 14, 16],
        'clf__learning_rate': [1, 1e-1, 1e-2, 0.3],
        'clf__booster': ['gbtree','dart'],
        #'n_estimators' : [300,500,600],
}

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(temp['Review_Text'], temp['Sentiment'], test_size=0.33, random_state=42)

In [ ]:
# from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

# score = 'f1_macro'
# print("# Tuning hyper-parameters for %s" % score)
# print()
# np.errstate(divide='ignore')
# clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=score)
# clf.fit(x_train, y_train)

# print("Best parameters set found on development set:")
# print()
# print(clf.best_params_)
# print()
# print("Grid scores on development set:")
# print()
# for mean, std, params in zip(clf.cv_results_['mean_test_score'], 
#                              clf.cv_results_['std_test_score'], 
#                              clf.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
# print()

# print("Detailed classification report:")
# print()
# print("The model is trained on the full development set.")
# print("The scores are computed on the full evaluation set.")
# print()
# print(classification_report(y_test, clf.predict(x_test), digits=4))
# print()
# print ("Confusion Matrix")
# print(confusion_matrix(y_test, clf.predict(x_test)))
# print("Accuracy Score")
# print(accuracy_score(y_test, clf.predict(x_test)))

A total number of combinations for the set of parameters above is a product of options for each parameter (3 x 5 x 3 x 3 x 3 = 405). It also needs to be multiplied by 5 to calculate a total number of data-fitting runs as we will be doing 5-fold cross-validation. That gets to be a large number in a hurry if you are using many parameters and lots of options, which is why brute-force grid search takes a long time.

Next we set up our classifier. We use sklearn's API of XGBoost as that is a requirement for grid search (another reason why Bayesian optimization may be preferable, as it does not need to be sklearn-wrapped). You should consider setting a learning rate to smaller value (at least 0.01, if not even lower), or make it a hyperparameter for grid searching. I am not using very small value here to save on running time.

Even though we have 4 threads available per job on Kaggle, I think it is more efficient to do XGBoost runs on single threads, but instead run 4 parallel jobs in the grid search. It's up to you whether you want to change this.


Next we set up our stratified folds and grid search parameters. I am using AUC as a scoring function, but you can plug in a custom scoring function here if you wish. Grid search wil spawn 4 jobs running a single thread each. The param_comb parameter declares how many different combinations should be picked randomly out of our total (405, see above). I am doing only 5 here, knowing that it will not properly sample the parameter space. Definitely use a bigger number for param_comb.

You may want to increase/decrease verbosity depending on your preference.

Note that I have set the number of splits/folds to 3 in order to save time. You should probably put 5 there to get a more reliable result.

In [ ]:
# from timeit import Timer

folds = 10
param_comb = 10

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(text_clf, param_distributions=tuned_parameters, n_iter=param_comb, n_jobs=4, cv=skf.split(x_train, y_train), verbose=3, random_state=1001 )

# # Here we go
# start_time = Timer(None) # timing starts from this point for "start_time" variable
random_search.fit(x_train, y_train)
# Timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best score:')
print(random_search.best_score_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
# results = pd.DataFrame(random_search.cv_results_)
# results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
wo_hyp_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', XGBClassifier())])

wo_hyp_clf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, wo_hyp_clf.predict(x_test), digits=4))
print()
print ("Confusion Matrix")
print(confusion_matrix(y_test, wo_hyp_clf.predict(x_test)))
print("Accuracy Score")
print(accuracy_score(y_test, wo_hyp_clf.predict(x_test)))


{'vect__ngram_range': (1, 2), 'tfidf__use_idf': False, 'tfidf__norm': 'l2', 'clf__subsample': 1.0, 'clf__min_child_weight': 5, 'clf__max_depth': 5, 'clf__gamma': 0.5, 'clf__colsample_bytree': 0.6}

In [ ]:
best_clf = Pipeline([('vect', CountVectorizer(ngram_range= (1, 2))),
                     ('tfidf', TfidfTransformer(use_idf= False, norm= 'l2')),
                     ('clf', XGBClassifier(use_label_encoder=False,
                                  eval_metric='mlogloss', subsample= 1.0, min_child_weight= 1, max_depth= 4, learning_rate = 1, gamma= 0.5, colsample_bytree= 0.6, booster = 'gbtree'))])
best_clf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, best_clf.predict(x_test), digits=4))
print()
print ("Confusion Matrix")
print(confusion_matrix(y_test, best_clf.predict(x_test)))
print("Accuracy Score")
print(accuracy_score(y_test, best_clf.predict(x_test)))


## 4. Conslusion
The model, which was trained on the development set, demonstrated $F_1=0.765$ on the evaluation set.

## References
1. Y. Rubtsova, "Constructing a Corpus for Sentiment Classification Training", Software & Systems, vol. 109, no. 1, pp. 72-78, 2015. 
2. "Naive Bayes", scikit-learn.org, 2018. [Online]. Available: http://scikit-learn.org/stable/modules/naive_bayes.html. [Accessed: 26- Aug- 2018]. 
3. "Working With Text Data", scikit-learn.org, 2018. [Online]. Available: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html. [Accessed: 26- Aug- 2018].